In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import sys  
sys.path.insert(0, 'object_tracking/source_code')
from object_detection import ObjectDetection
import math

# initialize object detection
od = ObjectDetection()

cap = cv2.VideoCapture('object_tracking/source_code/los_angeles.mp4')

#initialize count
count = 0
center_points_prev_frame = []

tracking_objects = {}
track_id = 0

while True:
    
    ret,frame = cap.read()
    count += 1
    
    if ret :
        
        #point current frame
        center_points_cur_frame = []
        
        # detect objects on frame
        (class_ids,scores,boxes) = od.detect(frame)
        for box in boxes:
            (x,y,w,h) = box
            cx = int((x + x + w) / 2)
            cy = int((y + y + h)/2)
            center_points_cur_frame.append((cx,cy))
            print("Frame N", count," ",x,y,w,h)
            
            
#             cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    
        # only at the beginning we compare previous and current frame
        if count <= 2:
            for pt in center_points_cur_frame:
                for pt2 in center_points_prev_frame:
                    distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])

                    if distance < 20:
                        tracking_objects[track_id] = pt
                        track_id += 1
        else:
            tracking_objects_copy = tracking_objects.copy()
            center_points_cur_frame_copy = center_points_cur_frame.copy()
            
            for object_id , pt2 in tracking_objects_copy.items():
                object_exists = False
                for pt in center_points_cur_frame_copy:
                    distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                    
                    # update IDS position
                    if distance < 20:
                        tracking_objects[object_id] = pt
                        object_exists = True
                        if pt in center_points_cur_frame:
                            center_points_cur_frame.remove(pt)
                            continue
                        
                # remove id if the object is no more in frame        
                if not object_exists:
                    tracking_objects.pop(object_id)
       
            #add new ids found
            for pt in center_points_cur_frame:
                tracking_objects[track_id] = pt
                track_id += 1
        
        for object_id , pt in tracking_objects.items():
            cv2.circle(frame,pt, 5, (0,0,255), -1)
            cv2.putText(frame,str(object_id), (pt[0], (pt[1] - 7)),0,1,(255,0,0), 1)
        
        print("tracking objects")
        print(tracking_objects)
    
        print("CUR FRAME LEFT PTS")
        print(center_points_cur_frame)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('Frame',frame)
        
        #make a copy of the points
        center_points_prev_frame = center_points_cur_frame.copy()
        
        key = cv2.waitKey(0)
        if key == 27:
            break
    else:
        print('no frame or the video ended')
        break
        
cap.release()
cv2.destroyAllWindows()